---
---
<h1 style="text-align: center; color: red;"> 🏆SPARK 3rd BraTS-Africa Brain Hackaton</h1>
<h2 style="text-align: center; color: black;">  Brain Tumor Segmentation</h2>


**<center> SPARK ACADEMY 2025 - DEEP LEARNING AND MEDICAL IMAGING  <center>**

<hr>
<hr>

**<center>General Informations<center>** 

*<center> Autor :  French Team <center>*

*<center>  Date : 13 May  2025  <center>*

---

## 🎯 **Overview**
In this hackaton, you worked with **3D MRI scans** from the **[BraTS2021 dataset](https://www.kaggle.com/datasets/dschettler8845/brats-2021-task1)** available on **Kaggle** . 
The goal of this competition is to tune SWIM UNETR models to make it more efficient for image segmentation


In [1]:
!pip -q install torchio monai medpy wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 4.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 52.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00

In [ ]:
import tarfile
import os

# Define file paths
input_tar_path = '/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar'
output_dir = '/kaggle/working/extracted_data'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Extract tar file
with tarfile.open(input_tar_path, 'r') as tar:
    tar.extractall(path=output_dir)

print(f"Extraction complete! Files are saved in: {output_dir}")


In [4]:
import os
import wandb
import numpy as np
import nibabel as nib
from glob import glob
from tqdm import tqdm
from statistics import mean
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchio as tio
from torchio.data import SubjectsLoader, SubjectsDataset

from monai.networks.nets import SwinUNETR
from monai.losses import DiceLoss, DiceCELoss

device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.__version__, device


<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-07-16 15:10:20.673504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752678621.030075      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752678621.134715      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


('2.6.0+cu124', device(type='cuda'))

In [5]:
from torch.utils.data import Dataset
import torchio as tio
import os
from glob import glob
import torch.nn.functional as F


NUM_CLASSES = 4  # Update this based on your actual classes

class LazySegmentationDataset(Dataset):
    def __init__(self, root_dir: str, transform=None, WT=False):
        self.transform = transform
        
        # Just store file paths
        self.flair_paths = sorted(glob(os.path.join(root_dir, "**", "*t2f*.nii"), recursive=True))
        self.t1_paths = sorted(glob(os.path.join(root_dir, "**", "*t1n.nii"), recursive=True))
        self.t1ce_paths = sorted(glob(os.path.join(root_dir, "**", "*t1c*.nii"), recursive=True))
        self.t2_paths = sorted(glob(os.path.join(root_dir, "**", "*t2w.nii"), recursive=True))
        self.mask_paths = sorted(glob(os.path.join(root_dir, "**", "*seg*.nii"), recursive=True))
        
        assert len(self.flair_paths) == len(self.t1_paths) == len(self.t1ce_paths) == len(self.t2_paths) == len(self.mask_paths), \
            "Mismatch in number of images"
        
        self.subject_paths = list(zip(self.flair_paths, self.t1_paths, self.t1ce_paths, self.t2_paths, self.mask_paths))
    
    def __len__(self):
        return len(self.subject_paths)
    


    def __getitem__(self, idx):
        flair_path, t1_path, t1ce_path, t2_path, mask_path = self.subject_paths[idx]
    
        subject = tio.Subject(
            flair=tio.ScalarImage(flair_path),
            t1=tio.ScalarImage(t1_path),
            t1ce=tio.ScalarImage(t1ce_path),
            t2=tio.ScalarImage(t2_path),
            mask=tio.LabelMap(mask_path),
        )
    
        if self.transform:
            subject = self.transform(subject)
    
        # 1. Remap label 4 to 3
        mask_tensor = subject['mask'][tio.DATA]  # [1, H, W, D]
        mask_tensor[mask_tensor == 4] = 3
        mask_tensor = mask_tensor.long()
    
        # 2. Remove channel dimension before one-hot encoding: [1, H, W, D] → [H, W, D]
        mask_tensor = mask_tensor.squeeze(0)
        if WT:
            mask_tensor[mask_tensor >=1] = 1
            NUM_CLASSES=2
            # 3. One-hot encode: [H, W, D] → [C, H, W, D]
            mask_onehot = F.one_hot(mask_tensor, num_classes=NUM_CLASSES).permute(3, 0, 1, 2).float()
        else:
            NUM_CLASSES=4
            mask_onehot = F.one_hot(mask_tensor, num_classes=NUM_CLASSES).permute(3, 0, 1, 2).float()
    
        # 4. Replace mask with one-hot version
        subject['mask'].set_data(mask_onehot)

        return subject

In [6]:
transforms = tio.Compose([
    tio.ToCanonical(),
    tio.ZNormalization(),
    tio.Resample((2.4, 2.4, 2.2)),
    tio.RescaleIntensity(out_min_max=(0, 1), percentiles=(0.05, 99.5)),
])

In [9]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
WT=False

full_dataset = LazySegmentationDataset('DATA PATH', transform=transforms,WT=WT)


# Define split lengths
total_size = len(full_dataset)
train_size = int(0.9 * total_size)
val_size = int(0.1 * total_size)
# test_size = total_size - train_size - val_size

# Optional: set manual seed for reproducibility
generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size], generator=generator
)

train_loader = SubjectsLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
val_loader   = SubjectsLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)
# test_loader  = SubjectsLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)


In [10]:
len(train_loader), len(val_loader)

(27, 3)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModalitySEAttention(nn.Module):
    def __init__(self, in_channels=4, reduction=2):
        super(ModalitySEAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)  # Global average pooling
        self.fc = nn.Sequential(
            nn.Conv3d(in_channels, in_channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels // reduction, in_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: (B, 4, H, W, D)
        attn = self.avg_pool(x)     # shape: (B, 4, 1, 1, 1)
        attn = self.fc(attn)        # shape: (B, 4, 1, 1, 1)
        return x * attn+x            # modality-wise scaling


In [12]:
class SwinUNetRMaskedAutoencoder1(nn.Module):
    def __init__(self, img_size=96, in_channels=4, patch_size=2):
        super().__init__()
        self.encoder = SwinUNETR(
            
            in_channels=in_channels,
            out_channels=4,  # Dummy output
            feature_size=48,
            use_checkpoint=False
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        # decoded = self.decoder(encoded)
        return encoded


In [15]:
Recon_model=SwinUNetRMaskedAutoencoder1(img_size=96, in_channels=4, patch_size=2)
Recon_model = torch.nn.DataParallel(Recon_model, device_ids=[0, 1])  # Use GPU 0 and GPU 1
Recon_model = Recon_model.to(device)
original=SwinUNETR(
            
            in_channels=4,
            out_channels=4,  
            feature_size=48,
            use_checkpoint=False
        )
# Recon_model.load_state_dict(torch.load("/kaggle/input/extended/pytorch/default/1/best_swinunetr_ssl_extended.pth"))

# Recon_model.module.encoder.out=original.out
Recon_model.load_state_dict(torch.load("/kaggle/input/best_2021_pretrain/pytorch/default/1/best_swinunetr_extended_20_epochs.pth"))
Recon_model=Recon_model.to(device)
# for name, param in Recon_model.module.named_parameters():
#     if name.startswith("enc."):  # Only allow decoder head to train
#         param.requires_grad = False

In [28]:
from torchinfo import summary
summary(Recon_model, input_size=
        (1, 4, 96, 96, 96))


Layer (type:depth-idx)                                            Output Shape              Param #
DataParallel                                                      [1, 4, 96, 96, 96]        --
├─SwinUNetRMaskedAutoencoder1: 1-1                                [1, 4, 96, 96, 96]        62,191,990
├─SwinUNetRMaskedAutoencoder1: 1-2                                --                        --
│    └─SwinUNETR: 2-1                                             [1, 4, 96, 96, 96]        62,191,990
│    └─SwinUNETR: 2-2                                             --                        --
│    │    └─SwinTransformer: 3-1                                  [1, 48, 48, 48, 48]       8,063,154
│    │    └─SwinTransformer: 3-2                                  --                        8,063,154
│    │    └─UnetrBasicBlock: 3-3                                  [1, 48, 96, 96, 96]       67,584
│    │    └─UnetrBasicBlock: 3-4                                  --                        67,584
│    │ 

In [ ]:
import numpy as np
import torch
from medpy.metric import binary as mpy

def compute_batch_metrics(logits, onehot_targets):
    pred = torch.argmax(logits, dim=1).cpu().numpy()          # [B, H, W, D]
    true = torch.argmax(onehot_targets, dim=1).cpu().numpy()  # [B, H, W, D]

    metrics = {
        'ET': {'dice': [], 'hd95': []},     # Label 3
        'NETC': {'dice': [], 'hd95': []},   # Label 1
        'SNTC': {'dice': [], 'hd95': []},   # Label 2
        'TC': {'dice': [], 'hd95': []},     # Labels 1+3
        'WT': {'dice': [], 'hd95': []},     # Labels 1+2+3
    }

    for b in range(pred.shape[0]):
        p, t = pred[b], true[b]

        masks = {
            'ET':   ((p == 3).astype(np.uint8), (t == 3).astype(np.uint8)),
            'NETC': ((p == 1).astype(np.uint8), (t == 1).astype(np.uint8)),
            'SNTC': ((p == 2).astype(np.uint8), (t == 2).astype(np.uint8)),
            'TC':   (((p == 1) | (p == 3)).astype(np.uint8), ((t == 1) | (t == 3)).astype(np.uint8)),
            'WT':   (((p > 0).astype(np.uint8)), ((t > 0).astype(np.uint8))),
        }

        for region, (pb, tb) in masks.items():
            intersection = (pb & tb).sum()
            union = pb.sum() + tb.sum()
            dice = 2 * intersection / union if union > 0 else 1.0 if pb.sum() == tb.sum() == 0 else 0.0

            try:
                hd95 = mpy.hd95(pb, tb) if pb.sum() and tb.sum() else 0.0
            except:
                hd95 = np.nan

            metrics[region]['dice'].append(dice)
            metrics[region]['hd95'].append(hd95)

    # Aggregate
    summary = []
    for region, values in metrics.items():
        summary.append({
            'Region': region,
            'Dice': round(np.nanmean(values['dice']), 4),
            'Hausdorff95': round(np.nanmean(values['hd95']), 4)
        })

    return summary


In [ ]:
from monai.losses import DiceLoss, TverskyLoss

# Initialize both losses
dice_loss = DiceLoss(to_onehot_y=False, softmax=True)
tversky_loss = TverskyLoss(to_onehot_y=False, softmax=True, alpha=0.3, beta=0.7)
ce_loss_fn = torch.nn.CrossEntropyLoss()

def combined_loss(logits, targets):
    return 0.7 * dice_loss(logits, targets) + 0.2 * tversky_loss(logits, targets)+0.1*ce_loss_fn(logits, targets)


In [ ]:
import torch
from tqdm import tqdm

def train_one_epoch_gard_acc(model, loader, optimizer):
    model.train()
    running_loss = 0.0
    accum_steps = 4
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(train_loader)):
        x = torch.cat([
            batch['flair'][tio.DATA],
            batch['t1'][tio.DATA],
            batch['t1ce'][tio.DATA],
            batch['t2'][tio.DATA],
        ], dim=1).to(device)

        y = batch['mask'][tio.DATA].to(device)  # One-hot encoded [B, C, H, W, D]

        logits = model(x)
        loss = combined_loss(logits, y)
        loss = loss / accum_steps
        loss.backward()
    
        if (i + 1) % accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        running_loss += loss.item()

    return running_loss / len(loader)


In [ ]:
import torch
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer):
    model.train()
    running_loss = 0.0

    for batch in tqdm(loader):
        x = torch.cat([
            batch['flair'][tio.DATA],
            batch['t1'][tio.DATA],
            batch['t1ce'][tio.DATA],
            batch['t2'][tio.DATA],
        ], dim=1).to(device)

        y = batch['mask'][tio.DATA].to(device)  # One-hot encoded [B, C, H, W, D]

        optimizer.zero_grad()
        logits = model(x)
        loss = combined_loss(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


In [ ]:
@torch.no_grad()
def validate(model, loader):
    model.eval()
    val_loss = 0.0
    all_logits = []
    all_targets = []

    for batch in tqdm(loader):
        x = torch.cat([
            batch['flair'][tio.DATA],
            batch['t1'][tio.DATA],
            batch['t1ce'][tio.DATA],
            batch['t2'][tio.DATA],
        ], dim=1).to(device)

        y = batch['mask'][tio.DATA].to(device)

        logits = model(x)
        loss = combined_loss(logits, y)
        val_loss += loss.item()

        all_logits.append(logits.cpu())
        all_targets.append(y.cpu())

    avg_val_loss = val_loss / len(loader)

    # Concatenate predictions and ground truth across batches
    all_logits = torch.cat(all_logits, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    metrics = compute_batch_metrics(all_logits, all_targets)  # list of dicts

    return avg_val_loss, metrics


In [ ]:
    avg_val_loss = val_loss / len(loader)



In [ ]:
# import wandb
# wandb.login(key="")

# wandb.init(project="", name="Swin_finetune_brats_v1", config={
#     "epochs": 100,
#     "batch_size": 2,
#     "img_size": (96, 96, 64),
#     "model": "SwinUNETR",
#     "loss": "Dice + Tversky+CE",
# })


In [ ]:
optimizer = torch.optim.AdamW(Recon_model.parameters(), lr=1e-5)
num_epochs = 100
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# Parameters for warm restarts
T_0 = 10          # Initial restart period
T_mult = 2        # Factor to increase restart period (10 -> 20 -> 40...)
eta_min = 1e-7    # Minimum learning rate

scheduler = CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=T_0, 
    T_mult=T_mult, 
    eta_min=eta_min
)

best_val_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch_gard_acc(Recon_model, train_loader, optimizer)
    val_loss, val_metrics = validate(Recon_model, val_loader)
    
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {current_lr:.6f}")
    
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "learning_rate": current_lr
    })
    
    for metric in val_metrics:
        region = metric['Region']
        wandb.log({
            f"{region}/dice": metric['Dice'],
            f"{region}/hd95": metric['Hausdorff95'],
        })
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': Recon_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_loss': best_val_loss,
            'val_metrics': val_metrics
        }, "best_swinunetr_warm_restart_v0.pth")
